# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** 

Classification : Classification is identifying a category to which a new observation belongs to, on the basis of a past observations. A classifier can predict a group/category to which an observation belongs to. The output is discrete/categorical value. A simple example for a classification would be finding if an animal is herbivorous or omnivorous or carnivorous given list of food it ate during the period of 10 days.

Regression : Regression analysis is a statistical process for estimating the relationships among variables. A regression can be used to estimate the output value based on the observation. The output value is continuous. A simple example for regression would be predicting the stock price based its history.

In the given problem goal is to build a model that predicts how likely a student is to pass their high school final exam, which is deciding between 2 categories (pass or fail). So it is a classfication problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] -1 

# TODO: Calculate passing students
n_passed = student_data[student_data["passed"] == "yes"].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data["passed"] == "no"].shape[0]

# TODO: Calculate graduation rate
grad_rate = float(n_passed)*100/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=11)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

#### Support Vector Machine (SVM):

   SVM techniques work the best when the data provided is linearly seperable. With the help of kernel functions in some cases SVM could also deal with non lineraly seperable data.[6,7] 

**Real-world applications:** 

***Cancer detection: ***

   Breast cancer classification is an area of active research. Several researchers are using different kinds of data to classify if the tumor is benign or malignant [1,2]. In [1] authors discuss about using non invasive imaging techniques and SVM to classify tumor with an accuracy of 88.10%, sensitivity and specificity of 85.71% and 90.48% respectively.


***Oil and Gas industry:***

   Several publications are available on proposed usage of SVM's in Oil and Gas industry[3,4,5]. Exploration of oil and gas is one such problem in which SVM's can be used. Exploration is the term used in the oil and gas industry to designate the search and recognition of structural or stratigraphic traps where hydrocarbons may have accumulated [3]. 
In [5] use of SVM to predict the distribution of hydrocarbons in a reservoir is discussed. In [4] the authors discuss about predicting (probably SVR's) porosity and permeability two important petrophysical properties which relate to amount of fluid in the proposed reservoir and their ability to flow. 


**Advantages:**
- SVM work very well when there is clear margin of seperation. Kernal functions can be used for non linearly seperable data
- SVM is memory efficient as it uses a small subset of training set in decision function
- Effective with high dimentional data

**Dis-advantages:**
- Training time is cubic in size of the dataset. So very slow when datasets are huge.
- Doesn't work well in the presence of noise generally prone to overfitting to the noise.
- SVM's do not provide probability estimates 


**Justification:** 
SVM's are better to be used with smaller datasets considering such as the dataset for this project their training time. The classification is memory efficient. Considering the number of features this problem is a multi dimentional classification problem and SVM's can be used for multi dimentional classification.

______________________________________________________________________________________________________________________

#### Stochastic Gradient Descent (SGD):
Stochastic gradient Descent is an optimazation method used in machine learning algorithms. SVM's, Linear regression classifiers can be trained using stochastic gradient method. Scikit-learn implementation of SGD is inspired from Stochastic Gradient SVM of Léon Bottou [8,9]. So SGD classifier has advantages of SVM along with improved efficency. 

**Real-world applications:** 
SGD's are used for training neural networks, deep learning networks, SVM's, linear regressors. SGD along with appropriate alogirthms can be used for solving classifcation or regression problems in different fields. Deep learning techniques can be used in facial recognition, image recognition, textual analysis, speech recogntion and many more applications.


**Advantages:**
- Advantages of SVM
- Computationally efficient in terms of training, which is basically linear with number of training examples. Computationally efficient beacuase, during training phase SGD uses a subset of training set instead of using entire training set to update its weights at each epoch.  
- Ease of implementation. There are lots of opportunities for code tuning.

**Dis-advantages:**
- Has a number of hyper-parameters
- SGD is sensitive for feature scaling
- prone to overfitting to the noise.
- do not provide probability estimates 

**Justification:** 
Considering the implentation of SGD in Scikit-learn I expect the results to be similar to SVM with improved efficiency.

______________________________________________________________________________________________________________________

#### Adaboost
The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction [10,11]. Can use with data that is textual, numeric, discrete, etc. 

**Real-world applications:** 
In [12] authors describe using ada boost algorithm for detection of basketball player from image frames. In the publication the authors describe speed is the reason why they choose adaboost but it didnot perform well. They suggested using Adaboost with some other method to classify the false positives of ada boost predictions to increase overall performance.

**Advantages:**
- Ease of implementation
- less parameters to tune 
- flexible — can combine with any learning algorithm
- In some problems it can be less prone to over fitting

**Dis-advantages:**
- AdaBoost classifier is combination many  weak learners and inaccurate classifiers, so the overall performace depends on performances of each learner/classifier.[13]
- suceptable to noise 

**Justification:** 
Several weak learners contribute towards final predication in Adaboost. The prediction can be a bit memory intensive and slow but due to aggregated result from weak classifiers I expect it to have better accuracy in prediction.

______________________________________________________________________________________________________________________

**References:**

[1] Thermography based breast cancer detection using texture features and Support Vector Machine by Acharya UR1, Ng EY, Tan JH, Sree SV.

[2] Application of SVM classifier in thermographic image classification for early detection of breast cancer by Witold Oleszkiewicz ; Paweł Cichosz ; Dariusz Jagodziński ; Mateusz Matysiewicz ; Łukasz Neumann ; Robert M. Nowak ; Rafał Okuniewski

[3] Applications of Support Vector Machines in the Exploratory Phase of Petroleum and Natural Gas: a Survey by Yuri Ccoicca

[4] Prediction of Oil and Gas Reservoir Properties using Support Vector Machines by Fatai Anifowose, AbdulAzeem Ewenla,Safiriyu Ijiyemi Eludiora

[5] Application of Support Vector Machine Method for Predicting Hydrocarbon in the Reservoir

[6] https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms

[7] http://scikit-learn.org/stable/modules/svm.html

[8] http://scikit-learn.org/stable/modules/sgd.html

[9] http://leon.bottou.org/projects/sgd

[10] http://scikit-learn.org/stable/modules/ensemble.html

[11] https://en.wikipedia.org/wiki/AdaBoost

[12] Application of AdaBoost Algorithm in Basketball Player Detection by Mihajlo Pupin, Djure Djakovica bb

[13] http://www.nickgillian.com/wiki/pmwiki.php/GRT/AdaBoost




### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    print clf
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn import linear_model

# TODO: Initialize the three models
clf_A = SVC(random_state=53)
clf_B = linear_model.SGDClassifier(random_state=40)
clf_C = AdaBoostClassifier(random_state=30)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
print "xxxxxxxxxxxxxxxxxxxxxxx SVM xxxxxxxxxxxxxxxxxxxxxxxxxxx"
print "100 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "200 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "300 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)


print "xxxxxxxxxxxxxxxxxxxxxxx SGD xxxxxxxxxxxxxxxxxxxxxxxxxxx"
print "100 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "200 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "300 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)



print "xxxxxxxxxxxxxxxxxxxxx ADABOOST xxxxxxxxxxxxxxxxxxxxxxxxx"
print "100 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "200 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "300 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

xxxxxxxxxxxxxxxxxxxxxxx SVM xxxxxxxxxxxxxxxxxxxxxxxxxxx
100 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Training a SVC using a training set size of 100. . .
Trained model in 0.0017 seconds
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=53, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0008 seconds.
F1 score for training set: 0.8684.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7945.
200 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Training a SVC using a training set size of 200. . .
Trained model in 0.0030 seconds
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=53, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0018 seconds.
F1 score for training set: 0.8758.
Made predic

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0017         |        0.0009          |  0.8684          |  0.7945         |
| 200               |        0.0030           |         0.0009         |    0.8758       |   0.8235        |
| 300               |        0.0058           |         0.0013         |    0.8596        |    0.8344      |

** Classifer 2 - SGD**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0005            |    0.0001          |     0.8212      |       0.7465    |
| 200               |     0.0005             |    0.0001             |   0.8418         |   0.8077        |
| 300               |     0.0006             |    0.0001              |  0.2222          |     0.2162     |

** Classifer 3 - AdaBoost**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0732              |      0.0035           |    0.9565       |    0.6829       |
| 200               |     0.0813              |      0.0036            |   0.8980         |  0.7971         |
| 300               |     0.0816              |      0.0039            |   0.8463        |  0.7910         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Comparision of results from experiments

Training time: SGD is the winner 

    SGD Classifier took least time while AdaBoost took most time among the classifiers.
    
Prediction time: SGD is the winner 

    SGD Classifier took least time while AdaBoost took most time among the classifiers.
    
F1 Score (training set): AdaBoost is the winner

    SVM showed almost stable increase in F1 scores with increase of dataset size while AdaBoost F1 scores decreased at a higher rate with increase of dataset size, SGD's F1 score was unstable.
    
F1 Score (testing set): SVM is the winner

    SVM outperformed AdaBoost alorithm and SGD algorithm in terms of F1 score. SGD algorithms's F1 scores were unstable again.
    
The SVM classifier has achieved higher F1 score with testing set and lower training,prediction times when compared to Ada Boost. SGD was fastest to train and predict but the F1 scores are unstable when compared to other classifiers. AdaBoost had good F1 score for prediction on training set but SVM performed better and gave stable results on testing set. So SVM classifier is appropriate for given data as the prediction F1 score was good and takes resonable time for training and prediction. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Support Vector Machine's are around for a long time and are being used in many real world applications. There are tons of publications around applications of SVM's. SVM classifier is apt for classifying data with clear margin of seperation. SVM classifier is simple algorithm. During training phase SVM's calculate boundary of seperation dividing similar types of data instances from a given dataset. During prediction phase the side of boundary to which a data instance belongs to determines its class.

How it works:

A set of points which are lineraly seperable can be seperated by one or more hyperplanes as shown in figure below. These hyperplanes can be found using a technique called quadratic programming.

<center><img width="300";height="200";src="images/many_seperators.png"></center>
<center>Lots of possible hyperplanes for given set of points</center>

SVM's enables us finding optimal hyper plane which can be used for defining the decision function. Support vectors are the elements of the training set that would change the position of the dividing hyper plane if removed or moved. Support vectors lie on hyperplanes H1 and H2 as shown in the figure. The distance between two hyper planes H1 and H2 is called margin. SVM finds hyper planes H1 and H2 which have biggest margin possible for given set of data and finally calculates the optimal hyper plane which will be exaclty in the middle of H1 and H2. During prediction phase the side of optimal hyperplane to which a data instance belongs to determines its class.[1]

<center><img width="300";height="200";src="images/svm.png"></center>
<center>Optimal hyper plane seperating 2 different classes of data </center>

To use SVM's with non linearly seperable data we can use kernal trick. The figure belows shows an example of kernal trick. In this example the kernel function transformed the given set of points to another space in which the points are linearly seperable. The function used for tranformantion is called kernel function. The kernel function used in the example is Radial basis fucntion (RBF). RBF, polynomial and sigmoidal kernels are some of the most used kernels.

<center><img width="700";height="100";src="images/kernel_trick.png"></center>
<center>Kernel trick example</center>



SVM's are memory efficient as it stores a small subset of training set (Support vectors) to make a prediction. It is effective even with datasets containing high number of features. It works well even if less number of data samples are provided as training set. In this project the size of dataset is less and there are many features, so this model is appropriate.


References:
[1] http://svms.org/tutorials/Berwick2003.pdf


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer 

# TODO: Create the parameters list you wish to tune
parameters = {'kernel': ['linear', 'poly', 'rbf'],
              'C':[1,2,3,10]}

# TODO: Initialize the classifier
#clf = linear_model.SGDClassifier()
clf = SVC(random_state=53)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters ,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print clf
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=53, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0041 seconds.
Tuned model has a training F1 score of 0.8596.
Made predictions in 0.0013 seconds.
Tuned model has a testing F1 score of 0.8344.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

   Prediction on training set = 0.8596
   
   Prediction on testing set = 0.8344

The tuned model was the same as untuned model even after using Grid CV for training the model using different configurations of parameters. So the F1 scores were exaclty same.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.